# Load in Libraries and Data

In [63]:
from tbats import TBATS, BATS
import pandas as pd
import numpy as np

import sklearn.metrics

In [25]:
# Load in and wrangle data
caiso_data = pd.read_csv("CAISO_data.csv")
print(caiso_data.shape)
caiso_data.tail()

(838260, 6)


,Unnamed: 0,timestamp,freq,market,ba_name,load_MW
838255,838255,2022-12-31 13:40:00+00:00,5m,RT5M,CAISO,19404.24
838256,838256,2022-12-31 12:15:00+00:00,5m,RT5M,CAISO,18772.96
838257,838257,2022-12-31 12:30:00+00:00,5m,RT5M,CAISO,18797.62
838258,838258,2022-12-31 10:55:00+00:00,5m,RT5M,CAISO,18811.79
838259,838259,2022-12-31 11:05:00+00:00,5m,RT5M,CAISO,18797.71


# Create Hourly Data and Train Test Split

In [42]:
hourly_data = caiso_data.copy()
hourly_data['date'] = pd.to_datetime(hourly_data['timestamp'].str[:13])
hourly_data = hourly_data.groupby('date').agg({'load_MW': 'max'}).reset_index()
hourly_data

,date,load_MW
0,2015-01-01 08:00:00,23041.00
1,2015-01-01 09:00:00,21973.00
2,2015-01-01 10:00:00,21294.00
3,2015-01-01 11:00:00,20805.00
4,2015-01-01 12:00:00,20729.00
...,...,...
70105,2023-01-01 03:00:00,25119.44
70106,2023-01-01 04:00:00,24266.00
70107,2023-01-01 05:00:00,23519.53
70108,2023-01-01 06:00:00,22592.39


In [80]:
# Hourly Train-Test Split

train_size = int(len(hourly_data) * 0.80)
hourly_train, hourly_test = hourly_data.iloc[:train_size], hourly_data.iloc[train_size:-1]
hourly_test

,date,load_MW
56088,2021-05-26 23:00:00,27419.48
56089,2021-05-27 00:00:00,28401.25
56090,2021-05-27 01:00:00,29026.08
56091,2021-05-27 02:00:00,29039.17
56092,2021-05-27 03:00:00,29163.87
...,...,...
70104,2023-01-01 02:00:00,26037.40
70105,2023-01-01 03:00:00,25119.44
70106,2023-01-01 04:00:00,24266.00
70107,2023-01-01 05:00:00,23519.53


# Create Daily Data and Train Test Split

In [44]:
daily_data = caiso_data.copy()
daily_data['date'] = pd.to_datetime(daily_data['timestamp'].str[:10])
daily_data = daily_data.groupby('date').agg({'load_MW': 'max'}).reset_index().iloc[1:-1,]
daily_data

,date,load_MW
1,2015-01-02,27451.00
2,2015-01-03,29090.00
3,2015-01-04,27641.00
4,2015-01-05,27752.00
5,2015-01-06,29781.00
...,...,...
2917,2022-12-27,26139.44
2918,2022-12-28,27667.84
2919,2022-12-29,27422.56
2920,2022-12-30,28035.32


In [69]:
# Daily Train-Test Split

train_size = int(len(daily_data) * 0.80)
daily_train, daily_test = daily_data.iloc[:train_size], daily_data.iloc[train_size:-1]
daily_test

,date,load_MW
2337,2021-05-26,29824.30
2338,2021-05-27,29163.87
2339,2021-05-28,29026.54
2340,2021-05-29,28804.58
2341,2021-05-30,26539.40
...,...,...
2916,2022-12-26,23856.77
2917,2022-12-27,26139.44
2918,2022-12-28,27667.84
2919,2022-12-29,27422.56


# Daily TBATS Model Fitting

In [46]:
# need to have a number for the length of each seasonal period for each 

estimator = TBATS(seasonal_periods=(7, 365.25))

In [47]:
model = estimator.fit(daily_train["load_MW"])

/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transformation/BoxCox.py:32: RuntimeWarning: overflow encountered in exp
  return np.exp(y)
/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transformation/BoxCox.py:32: RuntimeWarning: overflow encountered in exp
  return np.exp(y)
/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transformation/BoxCox.py:32: RuntimeWarning: overflow encountered in exp
  return np.exp(y)
/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transformation/BoxCox.py:32: RuntimeWarning: overflow encountered in exp
  return np.exp(y)
/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transformation/BoxCox.py:32: RuntimeWarning: overflow encountered in exp
  return np.exp(y)
/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transformation/BoxCox.py:32: RuntimeWarning: overflow encountered in exp
  return np.exp(y)
/Users/namanpatel/anaconda3/lib/python3.11/site-packages/tbats/transfo

In [68]:
# Training RMSE
np.sqrt(sklearn.metrics.mean_squared_error(model.y, model.y_hat))

1370.3458240602426

In [64]:
# Training MAPE
sklearn.metrics.mean_absolute_percentage_error(model.y, model.y_hat)

0.02917263156976322

In [73]:
y_forecast = model.forecast(steps=584)
y_forecast

array([30146.2622588 , 30337.03004252, 30292.27081845, 29779.4707431 ,
       28130.34134842, 29188.7882885 , 31275.89442162, 31416.99151345,
       31560.05906416, 31479.36053041, 30938.56097483, 29263.66253588,
       30316.79054267, 32408.24104211, 32547.85423639, 32692.34605976,
       32614.16700448, 32075.21905694, 30397.46946224, 31462.04259913,
       33570.15515125, 33717.034249  , 33868.66521401, 33796.22928815,
       33260.29621633, 31579.13766531, 32653.34476581, 34775.16910226,
       34925.31878233, 35079.27692727, 35007.08661072, 34468.11782098,
       32777.13667958, 33853.90304342, 35981.797279  , 36127.4662118 ,
       36275.71242487, 36195.52822171, 35645.19995174, 33936.27942868,
       35006.52209348, 37130.85443615, 37262.92911438, 37396.25549638,
       37298.89493384, 36728.20584665, 34992.961623  , 36046.87221865,
       38157.21551731, 38266.3467231 , 38375.44508281, 38251.80460896,
       37652.05647873, 35882.76844081, 36910.7695506 , 38996.86753746,
      

In [74]:
daily_test["load_MW"]

2337    29824.30
2338    29163.87
2339    29026.54
2340    28804.58
2341    26539.40
          ...   
2916    23856.77
2917    26139.44
2918    27667.84
2919    27422.56
2920    28035.32
Name: load_MW, Length: 584, dtype: float64

In [78]:
# Test RMSE
np.sqrt(sklearn.metrics.mean_squared_error(y_forecast, daily_test["load_MW"]))

2786.4678033601745

In [79]:
# Test MAPE
sklearn.metrics.mean_absolute_percentage_error(y_forecast, daily_test["load_MW"])

0.05953407899944143